In [ ]:
import subprocess
import glob
import pdb

In [ ]:
import warnings
import sys 
if not sys.warnoptions:
    warnings.simplefilter("ignore")

import os
import numpy as np
import pandas as pd
import scipy.io
from scipy import stats
from sklearn.manifold import MDS
import scipy.spatial.distance as sp_distance

import matplotlib.pyplot as plt
import seaborn as sns 
from mpl_toolkits.mplot3d import Axes3D

# %matplotlib notebook
%matplotlib inline
%autosave 5
sns.set(style = 'white', context='poster', rc={"lines.linewidth": 2.5})
sns.set(palette="colorblind")

In [110]:
test = '/Users/gracer/Documents/BBC_21-2939_prepro_data'

# Notes

Couldn't get subprocess to work so just used the bet via command line with the -F flag  
Bet wasn't working because google drive was stupid

bet input output -F

In [ ]:
subprocess.call(['bet','%s'%nifti,'%s'%output,'-F'])

Trying registration through FEAT. It works fine with the normal MNI brain, but is getting fucked with asym 
Fnirt is taking forever. Trying a different asym brain, maybe it is the skull. The cut off is really killing the registration. The brain is getting warped to try to be a whole brain with FNIRT. Trying a crisp BET and then FLIRT with the CIT MNI Asy brain no skull

Ok the regular is not gonna work. New plan
1. Register the standard brain to the T1
2. Use the details from the Stand2T1 to make ROI2T1
3. Then register the functional to the T1

All data needs to be in the subject space not the standard, since we are extracting the ROIs this sould be fine

In [ ]:
files = glob.glob(os.path.join(test, 'anat','ss_t1*.nii'))

In [ ]:
files[2]

In [ ]:
def get_t1(strc, basepath):
    T1 = strc
    sub = strc.split('/')[-1].split('.')[0].split('_')[-1]
    output_path = os.path.join(basepath,'derivatives','reg')
    return([sub,output_path,T1])

In [ ]:
def get_example_func(reference,T1,output_path,sub,style):
    example_func = '%s/%s_%s_example_func.nii.gz'%(output_path,sub,style)
    example_func_cmd = '/usr/local/fsl/bin/fslroi %s %s 300 1'%(reference,example_func)
    example_func_cmd_list= example_func_cmd.split(' ')
    subprocess.call(example_func_cmd_list)
    print(example_func)
    return(example_func)

In [ ]:
def func_thresh(example_func,output_path,sub):
    # threshold out extra bullshit
    print('thresholding')
    x = example_func.split('.')[0]
    x = '%s_thr'%x
    print(x)
    thr_cmd = '/usr/local/fsl/bin/fslmaths %s -thr 200 %s'%(example_func,x)
    thr_cmd_list = thr_cmd.split(' ')
    subprocess.call(thr_cmd_list)
    #example_func_thr = '%s/%s_example_func_thr.nii.gz'%(output_path,sub)
    example_func_thr = '%s.nii.gz'%x
    return(example_func_thr)

In [ ]:
def heavy_lifting(T1,example_func_thr, output_path, sub):
# register the T1 to the BOLD
    ## get the matrix
    print('welcome to the matrix')
    x = example_func.split('.')[0]
    outp = '%s_T12func.mat'%x
    cmd = 'flirt -in %s -ref %s -out outputvol -omat %s'%(T1,example_func_thr,outp)
    cmd_list = cmd.split(' ')
    subprocess.call(cmd_list)
    ## use the matrix
    print('neo use the matrix')
    matrix = outp
    y = '%s_T12func'%x
    flirt_cmd = 'flirt -in %s -ref %s -applyxfm -init %s -out %s'%(T1, example_func_thr,matrix,y)
    flrt_cmd_list = flirt_cmd.split(' ')
    subprocess.call(flrt_cmd_list)
    # Register the atlas image to the T1
    print('registering')
    T1_reg = y
    hi2std = '%s_highres2standard'%x
    hi2stdmat = '%s_highres2standard.mat'%x
    standard = os.path.join(test,'atlases','CIT168_Reinf_Learn_v1.1.0','MNI152-Nonlin-Asym-2009c','CIT168toMNI152-2009c_T1w_brain.nii.gz')
    s2t1_cmd = '/usr/local/fsl/bin/flirt -in %s -ref %s -out %s -omat %s -cost corratio -dof 12 -searchrx -90 90 -searchry -90 90 -searchrz -90 90 -interp trilinear'%(standard,T1_reg, hi2std,hi2stdmat)
    s2t1_cmd_list = s2t1_cmd.split(' ')
    subprocess.call(s2t1_cmd_list)
    return(hi2stdmat, T1_reg)

In [112]:
def roi_dir(mask_dir, sub, style):
    roi_out = os.path.join(mask_dir,'%s_%s'%(style,sub))
    try:
        os.makedirs(roi_out)
    except FileExistsError:
        pass
    return(roi_out)


In [113]:
def roi_create(roi_orig, roi_out, T1_reg, matrix):
    roi = roi_orig.split('/')[-1].split('_')[-1].split('.')[0]
    print(roi)
    ### align with the standard brain
    outp = '%s/%s_roi2T1'%(roi_out,roi)
    roi_cmd = 'flirt -in %s -ref %s -applyxfm -init %s -out %s'%(roi_orig,T1_reg,matrix,outp)
    print(roi_cmd)
    roi_cmd_list = roi_cmd.split(' ')
    subprocess.call(roi_cmd_list)
    return(outp, roi)

In [114]:
def roi_thresh(roi,roi_out,thresh):
    inp ='%s/%s_roi2T1.nii.gz'%(roi_out,roi)
    oot = '%s/%s_%s_roi2T1_thresh'%(roi_out,roi,thresh)
    print(thresh)
    thresh_cmd = '/usr/local/fsl/bin/fslmaths %s -thr %s %s'%(inp,thresh,oot)
    thresh_cmd_list = thresh_cmd.split(' ')
    subprocess.run(thresh_cmd_list)
    num = roi.split('l')[1]
    oupt = '%s/%s_%s_roi2T1_thresh_bin'%(roi_out,roi,thresh)
    bin_cmd = 'fslmaths %s -bin -mul %s %s'%(oot,num,oupt)
    bin_cmd_list = bin_cmd.split(' ')
    subprocess.run(bin_cmd_list)
    return(oupt)

In [115]:
for strc in glob.glob(os.path.join(test, 'anat','ss_t1*.nii')):
    sub = strc.split('/')[-1].split('.')[0].split('_')[-1]
    print(sub)
    for ref in glob.glob(os.path.join(test,'derivatives','BET','*%s*_brain.nii.gz'%sub)):
        style = ref.split('_')[4]
        mask_dir = os.path.join(test,'atlases','atlas')
        for x in glob.glob(os.path.join(mask_dir,'CIT168toMNI152_prob_atlas_bilat_1mm__vol*.nii.gz')):
            roi_orig = os.path.join(mask_dir,x)
            roi_out = roi_dir(mask_dir, sub,style)
            roi_path,roi = roi_create(roi_orig, roi_out, T1_reg, matrix)
            roithr = roi_thresh(roi,roi_out,'0.5')


20004


NameError: name 'T1_reg' is not defined

In [ ]:
roi = 'vol6'

In [ ]:
roi_thresh(roi,roi_out,'0.75')

In [ ]:
def roi_merger(path2rois, thresh):
    ## make a total brain
    rois = glob.glob(os.path.join(path2rois,'vol*_%s_roi2T1_thresh_bin.nii.gz'%thresh))
    finalgirl = os.path.join(path2rois,'total_rois')
    rois.append(finalgirl)
    roi_cmd2 = 'fslmaths %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s -add %s %s -odt input'%tuple(rois)
    roi_cmd2_list=roi_cmd2.split(' ')
    subprocess.run(roi_cmd2_list)


In [ ]:
for x in glob.glob(os.path.join(test,'atlases','atlas','*/')):
    print(x)
    roi_merger(x,'0.5')

In [ ]:
for strc in glob.glob(os.path.join(test, 'anat','ss_t1*.nii')):
    sub,output_path,T1 = get_t1(strc,test)
    print(sub)
    for ref in glob.glob(os.path.join(test,'derivatives','BET','*%s*_brain.nii.gz'%sub)):
        style = ref.split('_')[4]
        example_func = get_example_func(ref,T1,output_path,sub,style)
        example_func_thr = func_thresh(example_func,output_path,sub)
        matrix, T1_reg = heavy_lifting(T1, example_func_thr, output_path, sub)
        mask_dir = os.path.join(test,'atlases','atlas')
        for x in glob.glob(os.path.join(mask_dir,'CIT168toMNI152_prob_atlas_bilat_1mm__vol*.nii.gz')):
            roi_orig = os.path.join(mask_dir,x)
            roi_out = roi_dir(mask_dir, sub,style)
            roi_path,roi = roi_create(roi_orig, roi_out, T1_reg, matrix)
            roithr = roi_thresh(roi,roi_out,'0.5')


below here really didn't work, this is just to remind me not to do this again

In [ ]:
for struct in glob.glob(os.path.join(test,'anat','*.nii')):
    sub= struct.split('/')[6].split('_')[2].split('.')[0]
    print(sub)
    sub= os.path.join(test, 'anat',sub)
    cmd = 'fnirt --in=%s --aff=%s_my_affine_transf.mat --cout=%s_my_nonlinear_transf --config=T1_2_MNI152_2mm'%(struct,sub, sub)
    cmd_list = cmd.split(' ')
    #print(cmd_list)
    subprocess.call(cmd_list)






applywarp --ref=${FSLDIR}/data/standard/MNI152_T1_2mm --in=my_functional --warp=my_nonlinear_transf --premat=func2struct.mat --out=my_warped_functional

In [ ]:
reference_brain = '/Users/gracer/Documents/BBC_21-2939_prepro_data/MNI/mni_icbm152_t1_tal_nlin_asym_09c.nii'

This is taking an extremely long time, probably due to excess non-brain area

In [ ]:
for inp in glob.glob(os.path.join(test,'func','*','*_brain.nii.gz')):
    sub = inp.split('/')[7].split('_')[0].split('u')[1]
    trans = os.path.join(test, 'anat','%s_my_nonlinear_transf.nii.gz'%sub)
    bb = inp.split('/')[0:7]
    mat = os.path.join('/'.join(bb),'%s_struct2func.mat'%sub)
    outp = '%s_%s'%(inp.split('.')[0],'fnirt')
    cmd = 'applywarp --ref=%s --in=%s --warp=%s --premat=%s --out=%s -v'%(reference_brain, inp, trans, mat, outp)
    print(cmd)
    cmd_list = cmd.split(' ')
    subprocess.call(cmd_list)

Making masks

In [ ]:
fslmaths CIT168toMNI152_prob_atlas_bilat_1mm__vol1_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol2_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol3_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol4_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol5_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol6_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol7_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol8_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol9_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol10_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol11_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol12_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol13_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol14_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol15_mask.nii.gz -add CIT168toMNI152_prob_atlas_bilat_1mm__vol16_mask.nii.gz










In [ ]:
for roi in glob.glob(os.path.join(test,'atlases','atlas','*vol*.nii.gz')):
    print(roi)
    roiout=roi.split('.')[0]
    cmd = '/usr/local/fsl/bin/fslmaths %s -bin %s_mask'%(roi,roiout)
    cmd_list = cmd.split('/')
    subprocess.run(cmd_list)

fslmaths T1_brain -bin T1_new_brain_mask

# BIAC pipeline

Going to use the BIAC pipeline to get the timeseries into a dataframe  

https://wiki.biac.duke.edu/biac:analysis:resting_pipeline#download_source

I re-wrote it into more modern python conventions

In [1]:
import sys 
import os
import pandas as pd
import pdb
import glob

In [2]:
import numpy as np
import numpy.ma
import nibabel
from scipy import signal
import os, sys, subprocess
import string, random
import re
import networkx as nx
from optparse import OptionParser, OptionGroup
import logging
import math
from scipy import ndimage as nd
from scipy import stats
import scipy
print(scipy.__version__)
logging.basicConfig(format='%(asctime)s %(message)s ', datefmt='%m/%d/%Y %I:%M:%S %p', level=logging.INFO)


1.5.2


In [3]:
#corrlabel rois
def parcellator(basepath, func, corrlabel):
    print("starting parcellation")
    corrtxt = os.path.join(basepath,'corrlabel_ts.txt')
    cmd = 'fslmeants -i %s --label=%s -o %s'%(func, corrlabel, corrtxt)
    cmd_list = cmd.split(' ')
    print(cmd_list)
    subprocess.run(cmd_list)

/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/*

this is working :D so now just need to add in the insulin values and bin the corr into 1 minutes

In [40]:
def correlations(outpath, txtfile):
    print('starting correlation')
    rtxt = os.path.join(outpath,'new_r_matrix.csv')
    ztxt = os.path.join(outpath,'new_zr_matrix.csv')
    corrtxt = os.path.join(outpath,txtfile)
    
    if os.path.isfile(corrtxt):        
        timeseries = pd.read_csv(corrtxt, header=None,  skiprows=[0])
        timeseries = timeseries.drop([0], 1)
        timeseries = timeseries.apply(pd.to_numeric)
        myres = timeseries.corr(method='pearson')
        

#         zrmaps = stats.zscore(myres, nan_policy='omit')
        zrmaps = (myres - myres.mean())/myres.std()
        
        pd.DataFrame(zrmaps).to_csv(ztxt, index=False, header= False)
        pd.DataFrame(myres).to_csv(rtxt, index=False, header= False)





In [66]:
basepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base'


In [41]:
clamppath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp'


In [ ]:
for outpath in glob.glob(os.path.join(clamppath,'*')):
    sub = outpath.split('/')[-1]
    print(outpath)
    print(sub)
    txtfile = 'total_ts.csv'
    correlations(outpath, txtfile)

In [67]:
for outpath in glob.glob(os.path.join(basepath,'*/')):
    sub = outpath.split('/')[-1]
    print(outpath)
    print(sub)
    txtfile = 'total_ts.csv'
    correlations(outpath, txtfile)

/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/20001/

starting correlation
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/20002/

starting correlation
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/20003/

starting correlation
/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/base/20004/

starting correlation


In [12]:
basepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/'
for x in glob.glob(os.path.join(basepath,'*','*')):
    print(x)
    correlations(x)

/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp/20001


TypeError: correlations() missing 1 required positional argument: 'txtfile'

In [ ]:
outpath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output'
for func in glob.glob(os.path.join(test,'derivatives','BET','*_brain.nii.gz')):
    sub = func.split('/')[-1].split('_')[0].split('u')[-1]
    style = func.split('/')[-1].split('_')[1]
    print(sub)
    print(style)
    try:
        op = os.path.join(outpath,style,sub)
        os.makedirs(op)
    except FileExistsError:
        pass
    corrlabel = os.path.join('/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/atlas','%s_%s'%(style,sub),'total_rois.nii.gz')
    parcellator(op, func, corrlabel)


In [ ]:
test = '/Users/gracer/Documents/BBC_21-2939_prepro_data'

# Statistical analysis

In [58]:
labels = '/Users/gracer/Documents/BBC_21-2939_prepro_data/atlases/CIT168_Reinf_Learn_v1.1.0/labels.csv'

In [59]:
ROI_names = pd.read_csv(labels,sep = ',')

In [73]:
heads = list(ROI_names['ROI'])

In [74]:
heads

['Pu',
 'Ca',
 'NAC',
 'EXA',
 'GPe',
 'GPi',
 'SNc',
 'RN',
 'SNr',
 'PBP',
 'VTA',
 'VeP',
 'HN',
 'HTH',
 'MN',
 'STH']

In [61]:
def combo(path2ts, path2insulin, outpath, instatus):
    ts = pd.read_csv(path2ts, sep = ',', header = None)
    
    ts = ts.set_axis(heads, axis=1, inplace=False)
    ts['TR'] = ts.index
    ts['TR'] = pd.to_datetime(ts['TR'], unit='s')
    ts_downsamp = ts.set_index('TR').resample('60S').mean().dropna().reset_index()
    ts_downsamp = ts_downsamp.rename({'TR': 'time'}, axis=1)
    df = ts_downsamp
    
    if instatus == "clamp":
        in_ts = pd.read_csv(path2insulin, sep = ',', encoding='latin-1')
        in_ts['time'] = pd.to_datetime(in_ts['time'], unit='m')

        df = pd.merge(df, in_ts, on='time', how='outer')
        df = df.drop(['label','sex','pid'], 1)
        
    fp = outpath
    df.to_csv(fp, index=False)
    
    
    
    
    

In [77]:
x = '/Users/gracer/Documents/BBC_21-2939_insulin/20003.csv'
y = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp/20003/corrlabel_ts.csv'
z = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output/clamp/20003/test.csv'

In [91]:
df_corr = pd.read_csv(y, sep = ',', header = None)

In [96]:
df_corr

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,TR
0,409.998211,443.877005,437.333333,503.285714,405.216783,383.597222,428.4,463.966102,418.914286,0,0,399.2,404.5,489.110170,540.000000,427.058824,1
1,409.915027,441.442781,435.304348,501.214286,406.769231,385.277778,429.8,468.254237,418.942857,0,0,399.8,404.0,484.652542,537.571429,426.941176,2
2,409.467800,438.878075,436.855072,499.357143,407.020979,384.986111,429.0,463.135593,419.828571,0,0,401.6,402.0,485.593220,543.857143,427.411765,3
3,409.692308,439.376471,436.333333,494.785714,406.622378,384.750000,426.6,462.610170,418.657143,0,0,402.0,402.5,480.694915,534.000000,426.882353,4
4,410.463327,440.003209,436.913043,497.714286,407.958042,385.388889,427.4,464.000000,419.371429,0,0,403.8,404.0,485.042373,536.142857,428.647059,5
5,410.226297,440.544385,437.695652,501.714286,407.601399,385.166667,427.2,465.016949,418.542857,0,0,403.8,406.0,485.211864,536.000000,428.294118,6
6,410.497317,439.098396,438.913043,508.142857,408.237762,386.680556,430.0,465.644068,419.228571,0,0,405.0,402.5,487.474576,538.857143,427.235294,7
7,411.643113,439.127273,438.449275,503.142857,409.027972,386.680556,428.2,466.288136,419.428571,0,0,403.8,404.0,486.847458,537.714286,427.764706,8
8,413.711986,439.587166,438.304348,501.857143,411.265734,387.666667,429.8,465.677966,421.142857,0,0,404.4,402.5,486.779661,541.428571,428.529412,9
9,414.167263,441.003209,440.289855,503.857143,410.916084,388.138889,428.2,464.576271,419.371429,0,0,406.0,404.5,489.093220,539.857143,428.823529,10


In [93]:
df_corr['TR'] = df_corr.index
    

In [95]:
df_corr['TR'] = df_corr['TR']+1

In [98]:
df_corr['TR'] = pd.to_datetime(df_corr['TR'], unit='s')
df_corr_downsamp = df_corr.set_index('TR').resample('60S').mean().dropna().reset_index()

In [99]:
df_corr_downsamp

,TR,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,1970-01-01 00:00:00,409.311513,439.448636,435.260870,500.021792,406.148394,384.444444,427.406780,463.604711,418.066828,0,0,400.566102,402.228814,485.014651,536.794189,426.294118
1,1970-01-01 00:01:00,409.363238,439.836488,435.672705,500.521429,406.009091,384.017593,427.120000,463.481921,418.035238,0,0,401.163333,402.975000,485.759040,537.146429,426.172549
2,1970-01-01 00:02:00,409.208020,440.790660,435.012319,501.219048,405.537995,383.411574,426.280000,462.875989,417.371905,0,0,400.263333,402.166667,485.516525,536.951190,425.167647
3,1970-01-01 00:03:00,408.764595,440.627790,434.887198,501.214286,405.110839,382.658796,425.686667,462.300565,416.960952,0,0,399.993333,401.133333,484.918927,535.951190,424.723529
4,1970-01-01 00:04:00,409.366503,441.389822,435.250725,502.373810,405.338578,382.635880,426.070000,463.144068,417.221429,0,0,400.280000,401.316667,486.589689,538.922619,424.849020
5,1970-01-01 00:05:00,409.098450,441.576845,435.886715,502.844048,405.573776,382.592593,425.823333,462.862994,417.090476,0,0,401.283333,402.383333,486.126130,537.223810,424.486275
6,1970-01-01 00:06:00,409.390906,441.785365,436.160145,503.603571,405.725641,382.555093,425.883333,463.422316,417.235238,0,0,401.510000,403.375000,487.335311,538.511905,424.666667
7,1970-01-01 00:07:00,409.106202,441.587094,435.903140,502.692857,405.175991,381.943287,425.636667,463.239548,417.378571,0,0,400.926667,402.825000,486.700424,537.915476,424.694118
8,1970-01-01 00:08:00,408.921691,441.864617,436.127536,503.735714,405.104196,381.915046,425.690000,463.289266,417.106190,0,0,401.536667,404.016667,487.516808,538.054762,424.587255
9,1970-01-01 00:09:00,409.343143,442.339020,435.767391,504.275000,404.801515,381.371296,425.406667,463.439548,417.010000,0,0,400.983333,403.925000,487.940537,539.366667,424.288235


In [87]:
df_ins = pd.read_csv(x, sep = ',',  encoding='latin-1')

In [100]:
df_ins['time'] = df_ins['time'] -1 

In [102]:
df_ins['time'] = pd.to_datetime(df_ins['time'], unit='m')

In [103]:
df_ins

,pid,sex,label,insulin_uIU_mL,time,glucose_mg_dL
0,20003,Male,Insulin-endoÊ Ê Ê T1,5,1970-01-01 00:00:00,143
1,20003,Male,Insulin-endoÊ Ê Ê T2,5,1970-01-01 00:01:00,143
2,20003,Male,Insulin-endoÊ Ê Ê T3,5,1970-01-01 00:02:00,143
3,20003,Male,Insulin-endoÊ Ê Ê T4,5,1970-01-01 00:03:00,143
4,20003,Male,Insulin-endoÊ Ê Ê T5,6,1970-01-01 00:04:00,152
5,20003,Male,Insulin-endoÊ Ê Ê T6,5,1970-01-01 00:05:00,152
6,20003,Male,Insulin-endoÊ Ê Ê T7,6,1970-01-01 00:06:00,152
7,20003,Male,Insulin-endoÊ Ê Ê T8,6,1970-01-01 00:07:00,152
8,20003,Male,Insulin-endoÊ Ê Ê T9,5,1970-01-01 00:08:00,152
9,20003,Male,Insulin-endo Ê Ê T10,7,1970-01-01 00:09:00,152


In [107]:
df_corr_downsamp = df_corr_downsamp.rename({'TR': 'time'}, axis=1)
df = pd.merge(df_corr_downsamp, df_ins, on='time', how='outer')
df = df.drop(['label','sex','pid'], 1)

In [108]:
df

,time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,insulin_uIU_mL,glucose_mg_dL
0,1970-01-01 00:00:00,409.311513,439.448636,435.260870,500.021792,406.148394,384.444444,427.406780,463.604711,418.066828,0,0,400.566102,402.228814,485.014651,536.794189,426.294118,5.0,143.0
1,1970-01-01 00:01:00,409.363238,439.836488,435.672705,500.521429,406.009091,384.017593,427.120000,463.481921,418.035238,0,0,401.163333,402.975000,485.759040,537.146429,426.172549,5.0,143.0
2,1970-01-01 00:02:00,409.208020,440.790660,435.012319,501.219048,405.537995,383.411574,426.280000,462.875989,417.371905,0,0,400.263333,402.166667,485.516525,536.951190,425.167647,5.0,143.0
3,1970-01-01 00:03:00,408.764595,440.627790,434.887198,501.214286,405.110839,382.658796,425.686667,462.300565,416.960952,0,0,399.993333,401.133333,484.918927,535.951190,424.723529,5.0,143.0
4,1970-01-01 00:04:00,409.366503,441.389822,435.250725,502.373810,405.338578,382.635880,426.070000,463.144068,417.221429,0,0,400.280000,401.316667,486.589689,538.922619,424.849020,6.0,152.0
5,1970-01-01 00:05:00,409.098450,441.576845,435.886715,502.844048,405.573776,382.592593,425.823333,462.862994,417.090476,0,0,401.283333,402.383333,486.126130,537.223810,424.486275,5.0,152.0
6,1970-01-01 00:06:00,409.390906,441.785365,436.160145,503.603571,405.725641,382.555093,425.883333,463.422316,417.235238,0,0,401.510000,403.375000,487.335311,538.511905,424.666667,6.0,152.0
7,1970-01-01 00:07:00,409.106202,441.587094,435.903140,502.692857,405.175991,381.943287,425.636667,463.239548,417.378571,0,0,400.926667,402.825000,486.700424,537.915476,424.694118,6.0,152.0
8,1970-01-01 00:08:00,408.921691,441.864617,436.127536,503.735714,405.104196,381.915046,425.690000,463.289266,417.106190,0,0,401.536667,404.016667,487.516808,538.054762,424.587255,5.0,152.0
9,1970-01-01 00:09:00,409.343143,442.339020,435.767391,504.275000,404.801515,381.371296,425.406667,463.439548,417.010000,0,0,400.983333,403.925000,487.940537,539.366667,424.288235,7.0,152.0


In [ ]:
combo(y, x, z)

In [68]:
basepath = '/Users/gracer/Documents/BBC_21-2939_prepro_data/derivatives/output'

In [63]:
for sub_path in glob.glob(os.path.join(basepath,'*','*')):
#     print(sub_path)
    sub = sub_path.split('/')[-1]
    style = sub_path.split('/')[-2]
    print(style)
    path2ts = os.path.join(sub_path,'corrlabel_ts.csv')
    path2insulin = os.path.join('/Users/gracer/Documents/BBC_21-2939_insulin','%s.csv'%sub)
    outpath = os.path.join(sub_path,'total_ts.csv')
    combo(path2ts, path2insulin, outpath, style)

clamp
clamp
clamp
clamp
base
base
base
base


In [75]:
df = pd.read_csv(os.path.join(basepath,'clamp','20004','total_ts.csv'),header=None,  skiprows=[0])

In [76]:
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,1970-01-01 00:00:00,414.209705,434.357130,436.500242,528.386905,424.638578,417.444213,449.240000,456.845198,442.687143,0.0,0.0,413.933333,451.558333,506.252260,551.734524,437.002941,NaN,NaN
1,1970-01-01 00:01:00,414.033974,434.592353,436.407971,527.983333,423.965385,416.583333,447.953333,456.330791,441.861905,0.0,0.0,413.660000,451.108333,506.527260,552.046429,436.474510,4.0,154.0
2,1970-01-01 00:02:00,413.479040,435.093547,436.446377,529.973810,423.389977,415.603704,447.180000,455.862147,440.983333,0.0,0.0,413.363333,450.141667,506.744915,552.008333,435.738235,4.0,154.0
3,1970-01-01 00:03:00,413.743515,434.911622,436.472705,529.929762,423.576923,415.638657,448.080000,457.254237,441.520952,0.0,0.0,413.170000,450.916667,507.548023,552.814286,436.334314,3.0,154.0
4,1970-01-01 00:04:00,413.397287,434.944545,436.308937,529.104762,423.191725,415.031019,447.040000,457.501695,440.588095,0.0,0.0,413.020000,452.950000,506.564266,553.205952,435.720588,3.0,144.0
5,1970-01-01 00:05:00,412.930188,434.737772,436.816184,530.177381,422.914569,414.665046,446.833333,455.981356,440.139524,0.0,0.0,412.976667,451.425000,506.678814,550.473810,434.812745,4.0,144.0
6,1970-01-01 00:06:00,413.032797,435.064777,437.556039,531.003571,422.866783,414.302083,446.020000,455.413277,439.499048,0.0,0.0,413.240000,451.683333,507.951130,551.516667,433.946078,4.0,144.0
7,1970-01-01 00:07:00,412.777728,434.458699,437.155556,530.166667,422.352331,414.054861,445.810000,455.122599,439.390952,0.0,0.0,412.770000,451.025000,506.871045,548.742857,433.631373,4.0,144.0
8,1970-01-01 00:08:00,412.527609,434.057398,437.857729,529.214286,422.076573,413.891898,445.653333,454.566949,439.220952,0.0,0.0,413.233333,448.833333,507.564266,550.948810,433.699020,4.0,144.0
9,1970-01-01 00:09:00,412.080367,432.713458,438.810870,529.650000,421.728205,413.582639,444.603333,453.685028,438.959524,0.0,0.0,413.310000,448.500000,506.012712,547.125000,433.375490,4.0,155.0


In [ ]:
dataframe = dataframe.set_index("Date")
dataframe

In [ ]:
df = pd.read_csv(corrtxt, header=None,  skiprows=[1])


Plots

In [ ]:
# Plot

plt.plot(dataframe["Canada"], marker='o')

# Labelling 

plt.xlabel("Date")
plt.ylabel("Temp in Faherenheit")
plt.title("Pandas Time Series Plot")

# Display

plt.show()

start at step 7

resting_pipeline.py --func /path/to/run4.bxh --steps all --outpath /here/ -p func

Program to run through Nan-kuei Chen's resting state analysis pipeline:
    steps:
    0 - convert data to nii in LAS orientation ( we suggest LAS if you are skipping this step )
    1 - slice time correction
    2 - motion correction, then regress out motion parameter
    3 - skull stripping
    4 - normalize data
    5 - regress out WM/CSF
    6 - bandpass filter
    7 - do parcellation and produce correlation matrix from label file
      * or split it up:
         7a - do parcellation from label file
         7b - produce correlation matrix [--func option is ignored if step 7b
              is run by itself unless --dvarsthreshold is specified, and
              --corrts overrides default location for input parcellation
              results (outputpath/corrlabel_ts.txt)]
    8 - functional connectivity density mapping

–corrlabel
–corrtext

# Atlas

https://neurovault.org/images/56788/

In MNI152 2009c nlin asym space

Trying a thing where I register the atlas to normal MNI space

https://www.nature.com/articles/sdata201863

https://osf.io/r2hvk/